In [1]:
from fastai.vision.all import *

In [2]:
path = Path("/notebooks/tomatodiagnosis/test")

In [ ]:
path.ls()

In [ ]:
trn_path = path/'Tomato_mosaic_virus'

In [3]:
def train(arch, size, item=Resize(256, method='squish'), accum=1, finetune=True, epochs=10):
    dls = ImageDataLoaders.from_folder(path, valid_pct=0.2, item_tfms=item,
        batch_tfms=aug_transforms(size=size, min_scale=0.75), bs=64//accum)
    cbs = GradientAccumulation(64) if accum else []
    learn = vision_learner(dls, arch, metrics=error_rate, cbs=cbs).to_fp16()
    if finetune:
        learn.fine_tune(epochs, 0.01)
    else:
        learn.unfreeze()
        learn.fit_one_cycle(epochs, 0.01)

In [ ]:
train('convnext_small_in22k', 128, epochs=1, accum=1, finetune=False)

In [ ]:
import gc
def report_gpu():
    print(torch.cuda.list_gpu_processes())
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
report_gpu()

In [ ]:
train('convnext_small_in22k', 128, epochs=1, accum=2, finetune=False)
report_gpu()

In [ ]:
train('convnext_small_in22k', 128, epochs=1, accum=4, finetune=False)
report_gpu()

## Larger models

In [ ]:
train('convnext_large_in22k', 128, epochs=1, accum=2, finetune=False)
report_gpu()

In [ ]:
train('vit_large_patch16_224', 224, epochs=1, accum=4, finetune=False)
report_gpu()

In [ ]:
train('swinv2_large_window12_192_22k', 192, epochs=1, accum=2, finetune=False)
report_gpu()

In [ ]:
train('swin_large_patch4_window7_224', 224, epochs=1, accum=2, finetune=False)
report_gpu()

In [ ]:
train('swin_s3_base_224', 224, epochs=1, accum=2, finetune=False)
report_gpu()

## Training process

## Trying out swin transformer model

In [ ]:
path

In [ ]:
learn = train('swin_s3_base_224', 224, accum=2)

/usr/local/lib/python3.9/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


epoch,train_loss,valid_loss,error_rate,time
0,0.330888,0.441469,0.111509,06:32


epoch,train_loss,valid_loss,error_rate,time
0,0.115848,0.098238,0.031938,08:33
1,0.090191,0.042249,0.012665,08:33
2,0.095744,0.482325,0.088381,08:33
3,0.051221,0.040663,0.009912,08:33
4,0.059635,0.078413,0.017070,08:33
5,0.033369,0.036469,0.007985,08:36
6,0.020784,0.015684,0.003855,08:37
7,0.006657,0.015035,0.003304,08:33
8,0.006070,0.014865,0.003855,08:33


In [ ]:
learn.path = Path('.')

In [ ]:
learn.model.export('02_tomato.pkl')

In [ ]:
models = {
    'convnext_large_in22k': {
        (Resize(256), (128)),
    }, 'vit_large_patch16_224': {
        (Resize(256, method='squish'), 224),
        (Resize(256), 224),
    }, 'swinv2_large_window12_192_22k': {
        (Resize(256, method='squish'), 192),
        (Resize(256), 192),
    }, 'swin_large_patch4_window7_224': {
        (Resize(256), 224),
    }
}

In [ ]:
models = {
    'convnext_large_in22k': {
        (Resize(256), (128)),
    }, 'vit_large_patch16_224': {
        (Resize(256, method='squish'), 224),
        (Resize(256), 224),
    }, 'swinv2_large_window12_192_22k': {
        (Resize(256, method='squish'), 192),
        (Resize(256), 192),
    }, 'swin_large_patch4_window7_224': {
        (Resize(256), 224),
    }
}

In [ ]:
trn_path = path

In [ ]:
for arch,details in models.items():
    for item,size in details:
        print('---',arch)
        print(size)
        print(item.name)
        train(arch, size, item=item, accum=4, epochs=5)
        gc.collect()
        torch.cuda.empty_cache()